In [ ]:
import pandas as pd
import numpy as np
import os
from datetime import datetime

# Cargar archivo CSV original
df = pd.read_csv("../../data/raw/Neiva-co.csv")

# Renombrar columnas
df.rename(columns={
    "date": "Date", "tmax": "Tmax", "tmin": "Tmin", "tavg": "Tavg",
    "prcp": "Prcp", "wspd": "Wspd", "wdir": "Wdir"
}, inplace=True)

# Convertir columnas numéricas y rellenar vacíos con media redondeada
for col in ["Tmax", "Tmin", "Tavg", "Prcp", "Wspd"]:
    df[col] = pd.to_numeric(df[col], errors="coerce")
    df[col].fillna(round(df[col].mean(), 1),)
    df[col] = df[col].round(1)

# Convertir fecha a formato YYYY-MM-DD
df["Date"] = pd.to_datetime(df["Date"]).dt.date

# Convertir velocidad del viento de m/s a km/h
df["Wspd"] = (df["Wspd"] * 3.6).round(1)

# Asignar dirección del viento según fecha (Sur o Este)
def direccion_viento_neiva(fecha):
    md = (fecha.month, fecha.day)
    if (3, 24) <= md <= (10, 11):
        return "Sur"
    else:
        return "Este"

df["Wdir"] = df["Date"].apply(direccion_viento_neiva)

# Agregar columnas fijas
df["Estacion"] = "Neiva"
df["Provincia"] = "Huila"
df["Pais"] = "Colombia"

# Eliminar columnas irrelevantes si existen
df.drop(columns=["snow", "wpgt", "pres", "tsun"], inplace=True, errors="ignore")

# Reordenar columnas
df = df[["Date", "Tmax", "Tmin", "Tavg", "Prcp", "Wspd", "Wdir", "Estacion", "Provincia", "Pais"]]

# Guardar archivo limpio
salida = "../../data/cleaned/datos_limpios_neiva_2024.csv"
os.makedirs(os.path.dirname(salida), exist_ok=True)
df.to_csv(salida, index=False, encoding="utf-8")

print(f"Archivo limpio guardado en: {salida}")